In [1]:
from pydantic_ai import Agent
from toyaikit.tools import get_instance_methods

from search_tools import SearchTools

In [2]:
import logfire

In [3]:
import dotenv
dotenv.load_dotenv()

True

In [4]:
logfire.configure()  

In [5]:
logfire.instrument_pydantic_ai()

In [6]:
import requests
from minsearch import AppendableIndex

Logfire project URL: ]8;id=23745;https://logfire-eu.pydantic.dev/alexeygrigorev/ai-workshop-mlcon-berlin\https://logfire-eu.pydantic.dev/alexeygrigorev/ai-workshop-mlcon-berlin]8;;\

In [7]:
def initialize_index():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    documents = []

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            documents.append(doc) 

    index = AppendableIndex(
        text_fields=["question", "text", "section"],
        keyword_fields=["course"]
    )

    index.fit(documents)
    return index


index = initialize_index()
tools = SearchTools(index)

In [8]:
agent_tools = get_instance_methods(tools)
agent_tools

[<bound method SearchTools.add_entry of <search_tools.SearchTools object at 0x73d119f12510>>,
 <bound method SearchTools.search of <search_tools.SearchTools object at 0x73d119f12510>>]

In [9]:
agent_instructions = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

If you want to look up the answer, explain why before making the call. Use as many 
keywords from the user question as possible when making first requests.

Make multiple searches. Try to expand your search by using new keywords based on the results you
get from the search.

At the end, make a clarifying question based on what you presented and ask if there are 
other areas that the user wants to explore.
""".strip()

In [10]:
search_agent = Agent(
    name='search',
    model='openai:gpt-4o-mini',
    instructions=agent_instructions,
    tools=agent_tools,
)

In [11]:
result = await search_agent.run(user_prompt='how do I install kafka?')

14:52:21.676 search run
14:52:21.687   chat gpt-4o-mini
14:52:24.301   running 1 tool
14:52:24.301     running tool: search
14:52:24.319   chat gpt-4o-mini
14:52:26.421   running 1 tool
14:52:26.422     running tool: search
14:52:26.435   chat gpt-4o-mini
14:52:28.339   running 1 tool
14:52:28.340     running tool: search
14:52:28.353   chat gpt-4o-mini


In [24]:
print(result.output)

To install Apache Kafka, you typically go through several steps depending on whether you want it setup on a local machine, using Docker, or on a cloud service. Below is a general guide you can follow for a manual installation on a local machine. This process might involve a few prerequisites like Java, as Kafka is a Java application:

### Steps to Install Kafka Locally:

1. **Prerequisites**:
   - Ensure you have **Java** installed (Java 8 or later). 
   - You can check if Java is installed by running the command: `java -version` in your terminal.

2. **Download Kafka**:
   - Visit the [Apache Kafka download page](https://kafka.apache.org/downloads) and choose a version to download.
   - Extract the downloaded files.

3. **Start Zookeeper** (Kafka uses Zookeeper for managing distributed brokers):
   - Open your terminal and navigate to the Kafka directory:
     ```bash
     cd kafka_2.13-<version>
     ```
   - Then run Zookeeper using the command:
     ```bash
     bin/zookeeper-serve